# Download data in the region of interest using `Beacon`

__Notes:__
1. When getting data from Argo, we need to be sure that we have depth instead of pressure!7
2. We get more data when using `Beacon` than when using `pyargo`. Why is that?

In [ ]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
using PyPlot
const plt = PyPlot
using NCDatasets
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

## Map configuration (`cartopy` and directory)

In [ ]:
using Conda
Conda.add("cartopy")
using PyCall
ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
mpl = pyimport("matplotlib")
mpl.style.use("./fairease.mplstyle")
coast = cfeature.GSHHSFeature(scale="h")
datacrs = ccrs.PlateCarree();
figdir = "../figures/NorthAdriatic"
mkpath(figdir)

In [ ]:
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, footprintdir])

In [ ]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [ ]:
datasourcelist = ["Euro-Argo", "World Ocean Database", "SeaDataNet CDI TS", "CORA Profile", "CORA Timeseries"]
@info("Working on $(length(datasourcelist)) data sources")
domain = [12., 18., 43., 46.]
datestart = Dates.Date(1950, 1, 1)
dateend = Dates.Date(2024, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 500.
regionname = "NorthAdriaticSea"

## Loop on the data sources

In [ ]:
for datasource in datasourcelist
    
    @info("Working on $(datasource)")

    parameter1 = "TEMP"
    if datasource == "World Ocean Database"
        parameter1 = "Temperature"
    elseif datasource == "EMODnet Chemistry"  
        parameter1 = "ITS_90_water_temperature"
    elseif datasource == "SeaDataNet CDI TS"
        parameter1 = "ITS-90 water temperature"
    end
    parameter2 = "PSAL"
    
    @info("$(datestart) → $(dateend), $(parameter1)")
    
    # Download the "footprint"
    beaconURL = beacon_services[datasource]
    footprintURL = joinpath(beaconURL, "api/datasets/footprint")
    datasource_name = replace(datasource, " " => "_")
    footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
    @info("Footprint endpoint: $(footprintURL)")
    @info("Footprint file: $(footprintfile)")
    
    if isfile(footprintfile)
        @info("Footprint file already downloaded")
    else
        @info("Writing Footprint file")
        open(footprintfile, "w") do io
            r = HTTP.request("GET", footprintURL, 
                ["Authorization" => "Bearer $(token)"],
                response_stream=io)
            @info(r.status)
        end
    end
    
    data = JSON.parsefile(footprintfile);
    attributes = data["unique_column_attributes"]
    params = sort(collect(keys(attributes)));
    
    # Check if the parameters is in the list
    parameter1 in params ? @info("Parameter available") : @warn("Parameter not available")
    # @show(params);
    
    # Build query
    query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
        mindepth, maxdepth, minlon, maxlon, minlat, maxlat)
    
    # Construct file and figure names
    filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
    figname = "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.jpg"

    # Write the data in netCDF files
    @info("Data will be written in file:\n$(filename)")
    if isfile(filename)
        @info("File already downloaded")
        rm(filename)
    end
    @time open(filename, "w") do io
        r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
            ["Content-type"=> "application/json",
             "Authorization" => "Bearer $(token)"
            ],
            query, 
            response_stream=io);
        @info(r.status)
    end

    @info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")
    
    # Create plot
    plot_positions(filename, figname, datasource)
end   

In [ ]:
function plot_positions(filename::String, figname::String, datasource::String)
    NCDataset(filename, "r") do nc
        # print(nc)
        lon = nc["LONGITUDE"][:]
        lat = nc["LATITUDE"][:]

        fig = plt.figure()
        ax = plt.subplot(111, projection=ccrs.EuroPP())
        ax.plot(lon, lat, "go", ms=.5, transform=datacrs)
        ax.set_extent(domain)
        ax.add_feature(coast, lw=.25, color=".25")
        gl = ax.gridlines(crs=datacrs, draw_labels=true,
            linewidth=.5, color=".25", linestyle="--", zorder=2)
        gl.top_labels = false
        gl.right_labels = false
        ax.set_title("Temperature observations ($(length(lon))) in $(datasource)")
        plt.savefig(joinpath(figdir, figname))
        plt.close()

    end
    return nothing
end

In [ ]:
datasource = "CORA Timeseries"
datasource_name = replace(datasource, " " => "_")

@info("Working on $(datasource)")

parameter1 = "TEMP"
parameter2 = "PSAL"
    
    
beaconURL = beacon_services[datasource]
    
# Build query
query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
        mindepth, maxdepth, minlon, maxlon, minlat, maxlat)
    
filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
figname = "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.jpg"

# Write the data in netCDF files
@info("Data will be written in file:\n$(filename)")
if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-type"=> "application/json",
        "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end